In [1]:
!nvidia-smi

Wed Jun 14 09:00:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.20.3 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [4]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b',padding_size='left')
tokenizer.pad_token = tokenizer.eos_token

In [5]:
DEVICE = "cuda:0"
prompt = f"""
<human> : What is your return policy?
<assistant> :
""".strip()

encoding = tokenizer(prompt,return_tensors = 'pt').to(DEVICE)

In [6]:
encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

## QLoRA

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4', #noramlized float 4
    bnb_4bit_compute_dtype = torch.bfloat16, #binary float16
)

In [10]:
MODEL_ID = 'curiousily/falcon-7b-qlora-chat-support-bot-faq'
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             return_dict = True,
                                             quantization_config = bnb_config,
                                             device_map = 'auto',
                                             trust_remote_code = True)

model = PeftModel.from_pretrained(model,MODEL_ID)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [13]:
%%timeit -r 5

with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)

13.7 s ± 288 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [16]:
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)
print(tokenizer.decode(outputs[0],skip_special_tokens = True))

<human> : What is your return policy?
<assistant> : Our return policy allows you to return eligible items within 30 days of the purchase date for a full


## 4bit

In [5]:
MODEL_ID = 'curiousily/falcon-7b-qlora-chat-support-bot-faq'
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             return_dict = True,
                                             device_map = 'auto',
                                             trust_remote_code = True,
                                             load_in_4bit = True)

model = PeftModel.from_pretrained(model,MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [7]:
%%timeit -r 5

with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)

10 s ± 376 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [8]:
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)
print(tokenizer.decode(outputs[0],skip_special_tokens = True))

<human> : What is your return policy?
<assistant> : Our return policy allows you to return most items within 30 days of purchase. Please refer to our


## 8 bit

In [3]:
MODEL_ID = 'curiousily/falcon-7b-qlora-chat-support-bot-faq'
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             return_dict = True,
                                             device_map = 'auto',
                                             trust_remote_code = True,
                                             load_in_8bit = True)

model = PeftModel.from_pretrained(model,MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [9]:
%%timeit -r 5

with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)

2.89 s ± 47.5 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [10]:
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)
print(tokenizer.decode(outputs[0],skip_special_tokens = True))

<human> : What is your return policy?
<assistant> : Our return policy allows you to return most items within 30 days of purchase. Please refer to our


## Using Torch Compile

In [11]:
model = torch.compile(model)

In [12]:
%%timeit -r 5

outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)

3.07 s ± 98.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [13]:
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)
print(tokenizer.decode(outputs[0],skip_special_tokens = True))

<human> : What is your return policy?
<assistant> : Our return policy allows you to return most items within 30 days of purchase. Please refer to our


## Batched Inference

In [8]:
DEVICE = "cuda:0"

prompts = [
    f"""
    <Human> : What is return policy?
    <assisant> : """.strip(),
    f"""
    <Human> : How can I create an account?
    <assisant> : """.strip(),
    f"""
    <Human> : What payment method do you accept?
    <assisant> : """.strip(),
    f"""
    <Human> : What happens when I return a clearance item?
    <assisant> : """.strip(),
    ]

encoding = tokenizer(prompts,padding=True,truncation=True,return_tensors='pt').to(DEVICE)

In [9]:
MODEL_ID = 'curiousily/falcon-7b-qlora-chat-support-bot-faq'
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             return_dict = True,
                                             device_map = 'auto',
                                             trust_remote_code = True,
                                             load_in_8bit = True)

model = PeftModel.from_pretrained(model,MODEL_ID)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [11]:
%%timeit -r 5

with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

3.28 s ± 78.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [13]:
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids,attention_mask = encoding.attention_mask,generation_config = generation_config,do_sample = False,use_cache = True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [15]:
for out in outputs:
  print(tokenizer.decode(out,skip_special_tokens = True))

<Human> : What is return policy?
    <assisant> :# <a>: Can I return a product if it was purchased during a sale or with a
<Human> : How can I create an account?
    <assisant> :# Creating an account allows you to save your shipping and billing information for future purchases. You can also
<Human> : What payment method do you accept?
    <assisant> :#PaymentMethods

## Payment Methods

### Credit Card

We accept all major credit cards as
<Human> : What happens when I return a clearance item?
    <assisant> : If you return a clearance item, you will receive a refund for the item's original price minus


## Lit Parrot

In [1]:
!git clone https://github.com/Lightning-AI/lit-parrot
%cd lit-parrot/

Cloning into 'lit-parrot'...
remote: Enumerating objects: 1842, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 1842 (delta 517), reused 514 (delta 402), pack-reused 1140
Receiving objects: 100% (1842/1842), 606.90 KiB | 21.67 MiB/s, done.
Resolving deltas: 100% (1192/1192), done.
/content/lit-parrot


In [2]:
!git checkout a9dbfdf

Note: switching to 'a9dbfdf'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at a9dbfdf Use `init_module(empty_init)` (#131)


In [3]:
!pip install --index-url https://download.pytorch.org/whl/nightly/cu118 --pre 'torch>=2.1.0dev' -qqq --progress-bar off
!pip install huggingface_hub -r requirements.txt -qqq --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.1.0.dev20230613+cu118 which is incompatible.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0.dev20230613+cu118 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.1.0.dev20230613+cu118 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 2.1.0.dev20230613+cu118 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 2.1.0.dev20230613+cu118 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!python scripts/download.py --repo_id tiiuae/falcon-7b

Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]
Fetching 12 files:   8% 1/12 [00:00<00:05,  1.93it/s]
Fetching 12 files: 100% 12/12 [01:12<00:00,  6.05s/it]


In [7]:
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

2023-06-14 09:16:43.276145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Model config {'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True}
Processing checkpoints/tiiuae/falcon-7b/pytorch_model-00001-of-00002.bin
Processing checkpoints/tiiuae/falcon-7b/pytorch_model-00002-of-00002.bin


In [8]:
!python generate/base.py \
  --prompt "How can I create an account?"\
  --checkpoint_dir checkpoints/tiiuae/falcon-7b \
  --quantize llm.int8 \
  --max_new_tokens 20

2023-06-14 09:19:44.666381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True}
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
Time to instantiate model: 2.82 seconds.
Time to load the model weights: 117.93 seconds.
Global seed set to 1234
How can I create an account?
What is the My Invoices area?
What are the subscription plans?
How does
Time for inference 1: 3.54 sec total, 5.66 tokens/sec
Memory used: 9.72 GB
